# Dependencies

In [ ]:
!pip install swig
!pip install "gymnasium[box2d]" pygame matplotlib ipython

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
from IPython.display import HTML, Javascript, display

import base64
import io
from PIL import Image
import string


# CEM algorithm implementation

In [ ]:
class CEM:
    def __init__(
        self,
        dim: int,
        bounds: np.ndarray,
        population_size: int = 500,
        elite_frac: float = 0.1,
        max_iters: int = 50,
        smoothing: float = 0.7,
        seed: int = None,
    ):
        self.dim = dim
        self.bounds = bounds
        self.pop_size = population_size
        self.n_elite = max(1, int(np.ceil(elite_frac * population_size)))
        self.max_iters = max_iters
        self.alpha = smoothing
        self.rng = np.random.default_rng(seed)

        self.best_obj_history = []

    def optimize(self, x_init: np.ndarray, objective):
        mu = x_init.copy()
        sigma = (self.bounds[1] - self.bounds[0]) / 2.0

        self.best_obj_history = []

        for i in range(self.max_iters):
            pass

            # 1. random sample x and clip with bounds

            # 2. query y values from sampled x

            # 3. get top k minimum (x,y)

            # 3. calculate mean, std from top k samples

            # 3. update mean and sigma

            # [optional] For debug, log best y values and mean y values, etc ..

        # return final mean
        return mu

    def visualize(self):
        plt.figure()
        plt.plot(np.arange(1, len(self.best_obj_history) + 1), self.best_obj_history, marker='o')
        plt.title("Best Value per Iteration")
        plt.xlabel("Iteration")
        plt.ylabel("Objective (Lower is Better)")
        plt.show()

# Objective function

In [ ]:
def objective_vec(train_env, batch: np.ndarray) -> np.ndarray:
    """
    Generic vectorized discrete-action objective for CEM (to be MINIMIZED).
    Works for CartPole (action_dim=2) or LunarLander (action_dim=4) or any other discrete gym env.

    Parameters
    ----------
    train_env : VectorEnv
        A vectorized gymnasium environment.
    batch : np.ndarray, shape (N, D)
        Population of parameter vectors. Each θ has length
            obs_dim * action_dim  +  action_dim

    Returns
    -------
    np.ndarray, shape (N,)
        Negative average reward per candidate (CEM will minimize).
    """
    N, D = batch.shape
    obs_dim = train_env.single_observation_space.shape[0]
    action_dim = train_env.single_action_space.n

    # sanity check
    assert D == obs_dim * action_dim + action_dim, (
        f"Expected θ to be length {obs_dim*action_dim + action_dim}, "
        f"got {D}"
    )

    results = np.zeros(N, dtype=float)

    for i in range(N):
        theta = batch[i]
        # 1. unpack θ → W, b
        W = None
        b = None

        # 2. get sum of rewards with policy and env
        # Note: sum of rewards should be maximized, but CEM works to minimize objective. So we should change the sign.
        # gym env usage:
        # single step:
        #    obs, rewards, terminated, truncated, _ = train_env.step(actions)

    return results


## Helper function - evaluation

You don't need to implement this.

In [ ]:
def animate_policy(
    theta: np.ndarray,
    env_name: str = "CartPole-v1",
    max_steps: int = 500,
    interval: int = 100,
    goal_reward: float = None
) -> HTML:
    """
    Render a discrete-action policy for any Gymnasium env and display a reward curve
    with an optional goal_reward marker.

    Parameters
    ----------
    theta : np.ndarray, shape=(obs_dim*action_dim + action_dim,)
        Flattened policy parameters.
    env_name : str
        Gym environment ID.
    max_steps : int
        Max timesteps to run.
    interval : int
        Milliseconds between animation frames.
    goal_reward : float or None
        If provided, draw a horizontal line at this reward and a vertical line
        at the first step where cumulative reward ≥ goal_reward.
    """
    # Setup env
    env = gym.make(env_name, render_mode="rgb_array")
    obs_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    assert theta.size == obs_dim*action_dim + action_dim, \
        f"θ length should be {obs_dim*action_dim + action_dim}, got {theta.size}"
    W = theta[:obs_dim*action_dim].reshape(obs_dim, action_dim)
    b = theta[obs_dim*action_dim:]

    # Rollout and collect frames & rewards
    obs, _ = env.reset()
    frames, rewards = [], []
    total = 0.0
    goal_step = None

    for t in range(max_steps):
        frame = env.render()
        frames.append(frame)
        logits = obs.dot(W) + b
        action = int(np.argmax(logits))
        obs, r, terminated, truncated, _ = env.step(action)
        total += r
        rewards.append(total)
        if goal_reward is not None and goal_step is None and total >= goal_reward:
            goal_step = t+1
        if terminated or truncated:
            frames.append(env.render())
            rewards.append(total)
            break
    env.close()

    # Encode frames
    uris = []
    for f in frames:
        img = Image.fromarray(f)
        buf = io.BytesIO()
        img.save(buf, format="PNG")
        uris.append("data:image/png;base64," + base64.b64encode(buf.getvalue()).decode())

    # Plot reward curve with goal markers
    fig, ax = plt.subplots()
    ax.plot(range(len(rewards)), rewards, marker='o')
    if goal_reward is not None:
        ax.axhline(goal_reward, color='red', linestyle='--', label=f"Goal = {goal_reward}")
        if goal_step is not None:
            ax.axvline(goal_step, color='green', linestyle='--',
                       label=f"Reached at t={goal_step}")
        ax.legend()
    ax.set_title("Cumulative Reward")
    ax.set_xlabel("Step")
    ax.set_ylabel("Total Reward")
    plt.tight_layout()

    buf = io.BytesIO()
    fig.savefig(buf, format="PNG")
    plt.close(fig)
    graph_uri = "data:image/png;base64," + base64.b64encode(buf.getvalue()).decode()

    # Build HTML
    total_frames = len(frames)
    w = frames[0].shape[1]
    h = frames[0].shape[0]

    charset = string.ascii_letters + string.digits
    chars = np.array(list(charset))
    picks = np.random.choice(chars, size=6)
    # Join them into one string
    frame_id = ''.join(picks)


    html = f"""
    <div style="text-align:center;">
      <img id="cem_frame_{frame_id}" width="{w}" height="{h}" style="border:1px solid #333"/>
      <p id="frame_counter_{frame_id}">Frame: 1 / {total_frames}</p>
      <div>
        <img src="{graph_uri}" width="400"/>
      </div>
    </div>
    """
    display(HTML(html))

    js = f"""
    const frames = [{','.join(f"'{u}'" for u in uris)}];
    let idx = 0, total = frames.length;
    const img = document.getElementById('cem_frame_{frame_id}'),
          counter = document.getElementById('frame_counter_{frame_id}');
    img.src = frames[0];
    setInterval(() => {{
      idx = (idx + 1) % total;
      img.src = frames[idx];
      counter.innerText = 'Frame: ' + (idx+1) + ' / ' + total;
    }}, {interval});
    """
    display(Javascript(js))


# Optimization example : Cartpole
- [Gymnasium docs](https://gymnasium.farama.org/v0.29.0/environments/classic_control/cart_pole/)
- Takes about 30 sec ~ 1 min to optimize

In [ ]:
train_env = gym.make_vec(
    "CartPole-v1",
    num_envs = 3,
    vectorization_mode = "vector_entry_point"
)
obs_dim = train_env.single_observation_space.shape[0]
action_dim = train_env.single_action_space.n
objective = lambda batch: objective_vec(train_env, batch)

# You need to define
# 1. dimension of linear policy weight
# 2. bound where we should search policy
# 3. hyperparameters of CEM. population_size, elite_frace, max_iters, smoothing ratio etc...
# ----------------------------------------------------------------

dim = None
bounds = None
population_size = None
elite_frac = None
max_iters = None
smoothing = None
x0 = None

cem = CEM(dim=dim, bounds=bounds, population_size=population_size, elite_frac=elite_frac, max_iters=max_iters, smoothing=smoothing)

best_theta = cem.optimize(
    x0,
    objective,
)


Test if your policy is working well.

In [ ]:
animate_policy(best_theta, max_steps=500, interval=100, env_name='CartPole-v1', goal_reward=500)

# Optimization example : Lunar Lander
- [Gymnasium docs](https://gymnasium.farama.org/v0.29.0/environments/box2d/lunar_lander/)
- Takes about 10 min to optimize

In [ ]:
train_env = gym.make_vec(
    "LunarLander-v3",
    num_envs = 3,
)

# You need to define
# 1. dimension of linear policy weight
# 2. bound where we should search policy
# 3. hyperparameters of CEM. population_size, elite_frace, max_iters, smoothing ratio etc...
# ----------------------------------------------------------------

obs_dim = train_env.single_observation_space.shape[0]
action_dim = train_env.single_action_space.n
objective = lambda batch: objective_vec(train_env, batch)

# You need to define
# 1. dimension of linear policy weight
# 2. bound where we should search policy
# 3. hyperparameters of CEM. population_size, elite_frace, max_iters, smoothing ratio etc...
# ----------------------------------------------------------------

dim = None
bounds = None
population_size = None
elite_frac = None
max_iters = None
smoothing = None
x0 = None

cem = CEM(dim=dim, bounds=bounds, population_size=population_size, elite_frac=elite_frac, max_iters=max_iters, smoothing=smoothing)

best_theta = cem.optimize(
    x0,
    objective,
)

# ----------------------------------------------------------------


Test if your policy is working well.

In [ ]:
animate_policy(best_theta, max_steps=500, interval=100, env_name='LunarLander-v3', goal_reward=200)